<a href="https://colab.research.google.com/github/DylanCTY/TextAnalytics_LearningSpace/blob/main/IB9CW0_5504008_23*.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install the latest version of Ollama

In [1]:
# Install Ollama v0.1.40
!curl https://ollama.ai/install.sh | sed 's#https://ollama.ai/download#https://github.com/jmorganca/ollama/releases/download/v0.1.40#' | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10941    0 10941    0     0  29796      0 --:--:-- --:--:-- --:--:-- 29730>>> Downloading ollama...
100 10941    0 10941    0     0  26181      0 --:--:-- --:--:-- --:--:-- 26174
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


Get Ollama running in the background of the Colab

In [2]:
# Set up Ollama as a global variable and run it in the background
OLLAMA_MODEL='phi:latest'
import os
os.environ['OLLAMA_MODEL'] = OLLAMA_MODEL
!echo $OLLAMA_MODEL # Verify the global variable is set

import subprocess
import time

# Start ollama on the server ("serve")
command = "nohup ollama serve&" # "nohup" and "&" means run in the background

# Use subprocess.Popen to run the command
process = subprocess.Popen(command,
                            shell=True,
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE)

print("Process ID:", process.pid) # Print the process ID
time.sleep(5)  # Makes Python wait for 5 seconds

!ollama -v # print the Ollama version number as a check

phi:latest
Process ID: 729
ollama version is 0.1.41


Query the model to generate text about football games in the UK from 2004 to 2005

In [3]:
# Query the model via the command line
!ollama run $OLLAMA_MODEL "Briefly introduce the football games in the UK from 2004 to 2005."

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠇ pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling ma

Start to build our RAG. Firstly, this involves installing a bunch of packages:

In [4]:
# Install necessary packages
!pip install llama-index-embeddings-huggingface
!pip install llama-index-llms-ollama
!pip install llama-index-vector-stores-chroma
!pip install llama-index ipywidgets
!pip install llama-index-llms-huggingface
!pip install chromadb

# Import required modules from the llama_index library
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings, StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core.node_parser import SentenceSplitter, SemanticSplitterNodeParser
from llama_index.readers.file import FlatReader
from llama_index.core import Document
from llama_index.core.schema import TextNode
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

# Import the Ollama class
from llama_index.llms.ollama import Ollama

# Import ChromaVectorStore and chromadb module
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

# Additional imports
from zipfile import ZipFile
import os
from pathlib import Path
from transformers import AutoModel, AutoTokenizer
from numpy.linalg import norm
import torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
 

Preparing and preprocessing the dataset



In [5]:
# Unzip the dataset
zip_file_path = '/content/bbcsport_football.zip'  # Path to the uploaded zip file
unzip_dir = '/content/bbcsport_football'
with ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

# Combine the content of all text files into a single string, excluding hidden and system files
combined_text = ""
for root, dirs, files in os.walk(unzip_dir):
    for filename in sorted(files):
        if filename.endswith(".txt") and not filename.startswith('._') and not filename.startswith('__MACOSX'):
            with open(os.path.join(root, filename), 'r', encoding='utf-8', errors='ignore') as file:
                combined_text += file.read() + "\n"

# Replace "\n" (paragraph break) and "\t" (tab character)
combined_text = combined_text.replace("\n", "")
combined_text = combined_text.replace("\t", "")

# Write the combined text to a temporary file
temp_file_path = '/content/combined_bbcsport.txt'
with open(temp_file_path, 'w', encoding='utf-8') as temp_file:
    temp_file.write(combined_text)

# Read the combined text to verify its content
with open(temp_file_path, 'r', encoding='utf-8') as temp_file:
    content = temp_file.read()
    print(f"Combined text length: {len(content)} characters")
    print(f"First 1000 characters of the combined text:\n{content[:1000]}")

Combined text length: 516804 characters
First 1000 characters of the combined text:
Man Utd stroll to Cup winWayne Rooney made a winning return to Everton as Manchester United cruised into the FA Cup quarter-finals.Rooney received a hostile reception, but goals in each half from Quinton Fortune and Cristiano Ronaldo silenced the jeers at Goodison Park. Fortune headed home after 23 minutes before Ronaldo scored when Nigel Martyn parried Paul Scholes' free-kick. Marcus Bent missed Everton's best chance when Roy Carroll, who was later struck by a missile, saved at his feet.Rooney's return was always going to be a potential flashpoint, and he was involved in an angry exchange with a spectator even before kick-off. And Rooney's every touch was met with a deafening chorus of jeers from the crowd that once idolised the 19-year-old. Everton started brightly and Fortune needed to be alert to scramble away a header from Bent near the goal-line. But that was the cue for United to take complete co

We will use Llama Index's SimpleDirectoryReader to load all the documents from the combined dataset. After this, we will configure the Llama Index settings to use embedding models from Hugging Face. In this project, we will test two different chunking methods and two embedding models to evaluate their performance and effectiveness in our RAG system.

1-1. Semantic Chunking (model 1: BAAI/bge-small-en-v1.5)

In [ ]:
# Load the data from the combined text file
bbcsport_docs = FlatReader().load_data(Path(temp_file_path))

# Initialize a HuggingFace Embedding model
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

# Specify the embedding model into LlamaIndex's settings
Settings.embed_model = embed_model

# Initialize the SemanticSplitterNodeParser with desired settings
parser = SemanticSplitterNodeParser(
    buffer_size=2, breakpoint_percentile_threshold=93, embed_model=embed_model
)

# Get nodes using the semantic chunking model
bbcsport_nodes = parser.get_nodes_from_documents(bbcsport_docs)

# Convert nodes to the expected document type (TextNode)
docs = [TextNode(text=node.text, doc_id=str(i)) for i, node in enumerate(bbcsport_nodes)]

# Check if docs is defined and not empty
print(f"Total docs created: {len(docs)}")

# Print the first few docs to verify their content
for i in range(min(5, len(docs))):
    print(f"Doc {i+1} text:\n{docs[i].text}\n")

# Use the global variable (OLLAMA_MODEL) as our LLM
OLLAMA_MODEL = os.getenv('OLLAMA_MODEL', 'phi:latest')
llm = Ollama(model=OLLAMA_MODEL, request_timeout=900.0)

# Specify the LLM and embedding model into LlamaIndex's settings
Settings.llm = llm
Settings.embed_model = embed_model

Combined text length: 516804 characters
First 1000 characters of the combined text:
Man Utd stroll to Cup winWayne Rooney made a winning return to Everton as Manchester United cruised into the FA Cup quarter-finals.Rooney received a hostile reception, but goals in each half from Quinton Fortune and Cristiano Ronaldo silenced the jeers at Goodison Park. Fortune headed home after 23 minutes before Ronaldo scored when Nigel Martyn parried Paul Scholes' free-kick. Marcus Bent missed Everton's best chance when Roy Carroll, who was later struck by a missile, saved at his feet.Rooney's return was always going to be a potential flashpoint, and he was involved in an angry exchange with a spectator even before kick-off. And Rooney's every touch was met with a deafening chorus of jeers from the crowd that once idolised the 19-year-old. Everton started brightly and Fortune needed to be alert to scramble away a header from Bent near the goal-line. But that was the cue for United to take complete co

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Total nodes created: 402
Node 1 text:
Man Utd stroll to Cup winWayne Rooney made a winning return to Everton as Manchester United cruised into the FA Cup quarter-finals.Rooney received a hostile reception, but goals in each half from Quinton Fortune and Cristiano Ronaldo silenced the jeers at Goodison Park. Fortune headed home after 23 minutes before Ronaldo scored when Nigel Martyn parried Paul Scholes' free-kick. Marcus Bent missed Everton's best chance when Roy Carroll, who was later struck by a missile, saved at his feet.Rooney's return was always going to be a potential flashpoint, and he was involved in an angry exchange with a spectator even before kick-off. And Rooney's every touch was met with a deafening chorus of jeers from the crowd that once idolised the 19-year-old. Everton started brightly and Fortune needed to be alert to scramble away a header from Bent near the goal-line. But that was the cue for United to take complete control with a supreme passing display on a Good

1-2. Semantic Chunking (model 2: jinaai/jina-embeddings-v2-small-en)

In [ ]:
# Load the data from the combined text file
bbcsport_docs = FlatReader().load_data(Path(temp_file_path))

# Initialize a HuggingFace Embedding model
embed_model = HuggingFaceEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

# Specify the embedding model into LlamaIndex's settings
Settings.embed_model = embed_model

# Initialize the SemanticSplitterNodeParser with desired settings
parser = SemanticSplitterNodeParser(
    buffer_size=2, breakpoint_percentile_threshold=93, embed_model=embed_model
)

# Get nodes using the semantic chunking model
bbcsport_nodes = parser.get_nodes_from_documents(bbcsport_docs)

# Convert nodes to the expected document type (TextNode)
docs = [TextNode(text=node.text, doc_id=str(i)) for i, node in enumerate(bbcsport_nodes)]

# Check if docs is defined and not empty
print(f"Total docs created: {len(docs)}")

# Print the first few docs to verify their content
for i in range(min(5, len(docs))):
    print(f"Doc {i+1} text:\n{docs[i].text}\n")

# Use the global variable (OLLAMA_MODEL) as our LLM
OLLAMA_MODEL = os.getenv('OLLAMA_MODEL', 'phi:latest')
llm = Ollama(model=OLLAMA_MODEL, request_timeout=900.0)

# Specify the LLM and embedding model into LlamaIndex's settings
Settings.llm = llm
Settings.embed_model = embed_model

Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-small-en and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.2.intermediate.dense.bias', 'encoder.layer.2.intermediate.dense.weight', 'encoder.layer.2.output.LayerNorm.bias', 'encoder.layer.2.output.LayerNorm.weight', 'encoder.layer.2.output.dense.bias', 'encoder.layer.2.output.dense.weight', 'encoder.layer.3.intermediate.dense.bias', 'encoder.layer.3.intermediate.den

Total docs created: 282
Doc 1 text:
Man Utd stroll to Cup winWayne Rooney made a winning return to Everton as Manchester United cruised into the FA Cup quarter-finals.Rooney received a hostile reception, but goals in each half from Quinton Fortune and Cristiano Ronaldo silenced the jeers at Goodison Park. Fortune headed home after 23 minutes before Ronaldo scored when Nigel Martyn parried Paul Scholes' free-kick. Marcus Bent missed Everton's best chance when Roy Carroll, who was later struck by a missile, saved at his feet.Rooney's return was always going to be a potential flashpoint, and he was involved in an angry exchange with a spectator even before kick-off. And Rooney's every touch was met with a deafening chorus of jeers from the crowd that once idolised the 19-year-old. Everton started brightly and Fortune needed to be alert to scramble away a header from Bent near the goal-line. But that was the cue for United to take complete control with a supreme passing display on a Goodis

2-1. Sentence Splitter (model 1: BAAI/bge-small-en-v1.5)

In [6]:
# Define the TextNode class if not already defined
class TextNode:
    def __init__(self, text, doc_id):
        self.text = text
        self.doc_id = doc_id

# Load the data from the combined text file
bbcsport_docs = FlatReader().load_data(Path(temp_file_path))

# We will limit to chunk size 100
parser = SentenceSplitter(chunk_size=200, chunk_overlap=50)

# Get nodes using the sentence splitting model
bbcsport_nodes_raw = parser.get_nodes_from_documents(bbcsport_docs)

# Convert raw nodes to the expected document type (TextNode)
docs = [TextNode(text=node.text, doc_id=str(i)) for i, node in enumerate(bbcsport_nodes_raw)]

# Check if bbcsport_nodes is defined and not empty
print(f"Total nodes created: {len(bbcsport_nodes)}")

# Print the first few nodes to verify their content
for i in range(min(5, len(bbcsport_nodes))):
    print(f"Node {i+1} text:\n{bbcsport_nodes[i].text}\n")

# Initialize a HuggingFace Embedding model
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

# Use the global variable (OLLAMA_MODEL) as our LLM
OLLAMA_MODEL = os.getenv('OLLAMA_MODEL', 'phi:latest')
llm = Ollama(model=OLLAMA_MODEL, request_timeout=900.0)

# Specify the LLM and embedding model into LlamaIndex's settings
Settings.llm = llm
Settings.embed_model = embed_model

Total nodes created: 862
Node 1 text:
Man Utd stroll to Cup winWayne Rooney made a winning return to Everton as Manchester United cruised into the FA Cup quarter-finals.Rooney received a hostile reception, but goals in each half from Quinton Fortune and Cristiano Ronaldo silenced the jeers at Goodison Park. Fortune headed home after 23 minutes before Ronaldo scored when Nigel Martyn parried Paul Scholes' free-kick. Marcus Bent missed Everton's best chance when Roy Carroll, who was later struck by a missile, saved at his feet.Rooney's return was always going to be a potential flashpoint, and he was involved in an angry exchange with a spectator even before kick-off. And Rooney's every touch was met with a deafening chorus of jeers from the crowd that once idolised the 19-year-old.

Node 2 text:
And Rooney's every touch was met with a deafening chorus of jeers from the crowd that once idolised the 19-year-old. Everton started brightly and Fortune needed to be alert to scramble away a hea

2-2. Sentence Splitter (model 2: jinaai/jina-embeddings-v2-small-en)

In [33]:
# Define the TextNode class if not already defined
class TextNode:
    def __init__(self, text, doc_id):
        self.text = text
        self.id_ = doc_id
        self.hash = hash(self.text)  # Or any other appropriate way to generate a unique hash for the text

    def get_doc_id(self):
        return self.id_

    def get_metadata_str(self, mode=None):
        # This is a placeholder implementation. You can modify it based on your actual metadata requirements.
        return f"Document ID: {self.id_}"

    def get_content(self, metadata_mode=None):
        # Return the text content of the node
        return self.text

    def as_related_node_info(self):
        # This method is required for building relationships between nodes
        return {"node_id": self.id_, "hash": self.hash}

# Load the data from the combined text file
bbcsport_docs = FlatReader().load_data(Path(temp_file_path))

# We will limit to chunk size 100
parser = SentenceSplitter(chunk_size=200, chunk_overlap=50)

# Get nodes using the sentence splitting model
bbcsport_nodes_raw = parser.get_nodes_from_documents(bbcsport_docs)

# Convert raw nodes to the expected document type (TextNode)
docs = [TextNode(text=node.text, doc_id=str(i)) for i, node in enumerate(bbcsport_nodes_raw)]

# Check if bbcsport_nodes is defined and not empty
print(f"Total nodes created: {len(docs)}")

# Print the first few nodes to verify their content
for i in range(min(5, len(docs))):
    print(f"Node {i+1} text:\n{docs[i].text}\n")

# Initialize a HuggingFace Embedding model
embed_model = HuggingFaceEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

# Use the global variable (OLLAMA_MODEL) as our LLM
OLLAMA_MODEL = os.getenv('OLLAMA_MODEL', 'phi:latest')
llm = Ollama(model=OLLAMA_MODEL, request_timeout=900.0)

# Specify the LLM and embedding model into LlamaIndex's settings
Settings.llm = llm
Settings.embed_model = embed_model


Total nodes created: 862
Node 1 text:
Man Utd stroll to Cup winWayne Rooney made a winning return to Everton as Manchester United cruised into the FA Cup quarter-finals.Rooney received a hostile reception, but goals in each half from Quinton Fortune and Cristiano Ronaldo silenced the jeers at Goodison Park. Fortune headed home after 23 minutes before Ronaldo scored when Nigel Martyn parried Paul Scholes' free-kick. Marcus Bent missed Everton's best chance when Roy Carroll, who was later struck by a missile, saved at his feet.Rooney's return was always going to be a potential flashpoint, and he was involved in an angry exchange with a spectator even before kick-off. And Rooney's every touch was met with a deafening chorus of jeers from the crowd that once idolised the 19-year-old.

Node 2 text:
And Rooney's every touch was met with a deafening chorus of jeers from the crowd that once idolised the 19-year-old. Everton started brightly and Fortune needed to be alert to scramble away a hea

Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-small-en and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.2.intermediate.dense.bias', 'encoder.layer.2.intermediate.dense.weight', 'encoder.layer.2.output.LayerNorm.bias', 'encoder.layer.2.output.LayerNorm.weight', 'encoder.layer.2.output.dense.bias', 'encoder.layer.2.output.dense.weight', 'encoder.layer.3.intermediate.dense.bias', 'encoder.layer.3.intermediate.den

In [35]:
### TRIAL ###
# Load the data from the combined text file
bbcsport_docs = FlatReader().load_data(Path(temp_file_path))

# We will limit to chunk size 100
parser = SentenceSplitter(chunk_size=200, chunk_overlap=50)

# Get nodes using the sentence splitting model
bbcsport_nodes_raw = parser.get_nodes_from_documents(bbcsport_docs)

# Convert raw nodes to the expected document type (Document)
docs = [Document(text=node.text, doc_id=str(i)) for i, node in enumerate(bbcsport_nodes_raw)]

# Check if docs is defined and not empty
print(f"Total documents created: {len(docs)}")

# Print the first few documents to verify their content
for i in range(min(5, len(docs))):
    print(f"Document {i+1} text:\n{docs[i].text}\n")

# Initialize a HuggingFace Embedding model
embed_model = HuggingFaceEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

# Use the global variable (OLLAMA_MODEL) as our LLM
OLLAMA_MODEL = os.getenv('OLLAMA_MODEL', 'phi:latest')
llm = Ollama(model=OLLAMA_MODEL, request_timeout=900.0)

# Specify the LLM and embedding model into LlamaIndex's settings
Settings.llm = llm
Settings.embed_model = embed_model

Total documents created: 862
Document 1 text:
Man Utd stroll to Cup winWayne Rooney made a winning return to Everton as Manchester United cruised into the FA Cup quarter-finals.Rooney received a hostile reception, but goals in each half from Quinton Fortune and Cristiano Ronaldo silenced the jeers at Goodison Park. Fortune headed home after 23 minutes before Ronaldo scored when Nigel Martyn parried Paul Scholes' free-kick. Marcus Bent missed Everton's best chance when Roy Carroll, who was later struck by a missile, saved at his feet.Rooney's return was always going to be a potential flashpoint, and he was involved in an angry exchange with a spectator even before kick-off. And Rooney's every touch was met with a deafening chorus of jeers from the crowd that once idolised the 19-year-old.

Document 2 text:
And Rooney's every touch was met with a deafening chorus of jeers from the crowd that once idolised the 19-year-old. Everton started brightly and Fortune needed to be alert to scrambl

Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-small-en and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.2.intermediate.dense.bias', 'encoder.layer.2.intermediate.dense.weight', 'encoder.layer.2.output.LayerNorm.bias', 'encoder.layer.2.output.LayerNorm.weight', 'encoder.layer.2.output.dense.bias', 'encoder.layer.2.output.dense.weight', 'encoder.layer.3.intermediate.dense.bias', 'encoder.layer.3.intermediate.den

Check the output

In [8]:
# Access the second node's text
print(bbcsport_nodes[1].text)

And Rooney's every touch was met with a deafening chorus of jeers from the crowd that once idolised the 19-year-old. Everton started brightly and Fortune needed to be alert to scramble away a header from Bent near the goal-line. But that was the cue for United to take complete control with a supreme passing display on a Goodison Park pitch that was cutting up. Fortune gave United the lead after 23 minutes, rising to meet Ronaldo's cross from eight yards after the Portuguese youngster had been allowed too much time and space by the hapless Gary Naysmith. United dominated without creating too many clear-cut chances, and they almost paid the price for not making the most of their domination two minutes before half-time. Mikel Arteta played a superb ball into the area but Bent, played onside by Gabriel Heintze, hesitated and Carroll plunged at his fee to save.


Set up a vector database using Chroma DB and index our documents with embeddings. This will enable us to search for relevant documents based on our queries

In [36]:
# Create client ("db") and a database ("chroma_db")
db = chromadb.PersistentClient(path="./chroma_db")

# Create a collection/table ("demo-for-bbcsport") in the db
chroma_collection = db.create_collection("demo-for-bbcsport10")

# Set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
# Specify Chroma as our vector db
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create the index
index = VectorStoreIndex.from_documents(
    docs,  # the abstract documents (nodes from bbcsport_nodes)
    storage_context=storage_context,
    embed_model=embed_model
)

# Print the metadata
print(chroma_collection)

# Print the name of the collection (table)
print(f'Collection name is: {chroma_collection.name}')

BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


name='demo-for-bbcsport10' id=UUID('8c6a0e69-b126-47c3-bf10-c2891653a53a') metadata=None tenant='default_tenant' database='default_database'
Collection name is: demo-for-bbcsport10


Setting up and testing query engine with prompt templates

In [37]:
# Set up a prompt template to ensure the LLM always answers
qa_prompt_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question: {query_str}\n"
)

# Text QA Prompt
chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            "Always answer the question concisely, even if the context isn't helpful." # Add 'concisely'
        ),
    ),
    ChatMessage(role=MessageRole.USER, content=qa_prompt_str),
]

text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

# Initialize query engine with the prompt template and LLM
query_engine = index.as_query_engine(
    text_qa_template=text_qa_template,
    llm=llm,
)

# Sample queries for testing
queries_1 = [
    "What are Cristiano Ronaldo's thoughts on the contract renewal offer from Manchester United?", #From the news 004
    "Which Chelsea players have suffered serious injuries and are likely to miss multiple games?", #From the news 017
    "What are Jaap Stam's feelings towards Manchester United after they sold him in 2001?" #From the news 064
] # Selected manually from the dataset
queries_2 = [
    "Who won the crown of the Premier League in 2005?",
    "Describe the key players in Manchester United during the 2004-2005 season.",
    "How did Chelsea perform in the 2004-2005 football season?"
] # Generated by ChatGPT


# Execute and print responses for queries_1
print("Responses for queries_1:")
for query in queries_1:
    response = query_engine.query(query)  # query the model with context
    print(f"Query: {query}")
    print(f"Response: {response.response}\n")

# Execute and print responses for queries_2
print("Responses for queries_2:")
for query in queries_2:
    response = query_engine.query(query)  # query the model with context
    print(f"Query: {query}")
    print(f"Response: {response.response}\n")

Responses for queries_1:
Query: What are Cristiano Ronaldo's thoughts on the contract renewal offer from Manchester United?
Response:  There is no information in the given text to suggest what Cristiano Ronaldo's thoughts are on the contract renewal offer from Manchester United.


Query: Which Chelsea players have suffered serious injuries and are likely to miss multiple games?
Response:  The article does not provide specific details about which Chelsea players have suffered serious injuries and are likely to miss multiple games. It only states that senior stars Ryan Giggs and Roy Keane came on the bench for injured John O'Shea, and that they were able to bring composure and a threat to midfield, as well as penetration from Giggs. The article also mentions Didier Drogba being sidelined but does not give information about how serious his injury is or if he will be out multiple games.


Consider the following statements: 
1. If a player has suffered a serious injury they are likely to mi

Inspect the output metadata

In [38]:
response.metadata # print the full output from the LLM

{'4cd76ecc-1083-4495-a3f9-383911bc479c': {},
 '8dc2fe88-b37f-4d10-9c22-219192dffb61': {}}